In [ ]:
%pip install setuptools
%pip install ydata_profiling
%pip install scikit-learn
%pip install xlrd
%pip install openpyxl
%pip install ipywidgets
%pip install statsmodels
%pip install catboost

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from ydata_profiling import ProfileReport
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from statsmodels.tsa.stattools import adfuller
from catboost import CatBoostRegressor
import catboost
from catboost import *
import statsmodels.api as sm

In [ ]:
excel_data=pd.read_excel("././dataset/Объёмы перевозок.xls", skiprows=[0], header=[0,1])
excel_data_volumes = excel_data

In [ ]:
excel_data_volumes.sort_index(axis=1, inplace=True)
num_col=len(excel_data_volumes.columns)
left = excel_data_volumes.iloc[:, [num_col-5, num_col-4, num_col-3, num_col-2,num_col-1]]
left.columns = left.columns.get_level_values(1)
left = left.set_axis(["id", "subj_from", "subj_to", "cargo_code", "cargo_type"], axis = 1)

In [ ]:
labelencodder = LabelEncoder()
subj = pd.concat([left.loc[:, ["subj_from"]].rename(columns = {"subj_from": "subj"}), left.loc[:, ["subj_to"]].rename(columns = {"subj_to": "subj"})]).drop_duplicates(ignore_index=True)
cargo_type = left.loc[:, ["cargo_type"]].drop_duplicates(ignore_index=True)
subj.to_excel("././dataset/subj.xlsx")
cargo_type.to_excel("././dataset/cargo_type.xlsx")

In [ ]:
left.loc[:, ["subj_from"]] = left.loc[:, ["subj_from"]].apply(lambda x: subj[subj["subj"] == x["subj_from"]].index[0], axis=1)
left.loc[:, ["subj_to"]] = left.loc[:, ["subj_to"]].apply(lambda x: subj[subj["subj"] == x["subj_to"]].index[0], axis=1)
left.loc[:, ["cargo_type"]] = left.loc[:, ["cargo_type"]].apply(lambda x: cargo_type[cargo_type["cargo_type"] == x["cargo_type"]].index[0], axis=1)

In [ ]:
left["subj_from"] = left["subj_from"].astype("int64")
left["subj_to"] = left["subj_to"].astype("int64")
left["cargo_type"] = left["cargo_type"].astype("int64")

In [ ]:
for i in range(int((num_col-5)/2)):
    right = excel_data_volumes.iloc[:, [i*2,i*2+1]]
    date = pd.to_datetime(excel_data_volumes.iloc[:, [i*2]].columns.get_level_values(0).to_list()[0], format="%Y/%m")
    
    right.insert(2, ("", "date"), date)
    right.insert(3, ("", "month"), date.month)
    right.insert(4, ("", "quarter"), date.quarter)
    right.insert(5, ("", "year"), date.year)
    right.columns = right.columns.get_level_values(1)
    right = right.set_axis(["cargo_volume", "cargo_sales", "date", "month", "quarter", "year"], axis=1)
    right.insert(2, ("cargo_price"), right.apply(lambda x: (x["cargo_sales"] / x["cargo_volume"]) if x["cargo_volume"] > 0 else x["cargo_sales"], axis= 1))
    if i == 0:
        result = pd.concat([left, right], axis = 1, sort = False)
    else:
        result = pd.concat([result, pd.concat([left, right], axis = 1, sort = False)], axis = 0, ignore_index=True, sort=False)
excel_data_volumes = result

In [ ]:
excel_data_volumes.insert(12, ("month_of_period"), 0)

In [ ]:
min = excel_data_volumes["year"].min()
excel_data_volumes["month_of_period"] = excel_data_volumes["year"].apply(lambda x: x - min)*12 + excel_data_volumes["month"] 
excel_data_volumes = excel_data_volumes.drop("date", axis = 1)

In [ ]:
excel_data_volumes.to_excel("././dataset/volumes.xlsx")

In [ ]:
excel_data_volumes

In [ ]:
excel_data=pd.read_excel("././dataset/MC_1.xls", header=[0])
excel_data_MC = excel_data
excel_data=pd.read_excel("././dataset/MC_2.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_4.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_5.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_6.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_7.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_8.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)

In [ ]:
excel_data_MC = excel_data_MC.drop(excel_data_MC.columns[[3,4,6,7,8,13,15,17]], axis = 1).drop_duplicates()
excel_data_MC = excel_data_MC.set_axis(["id", "is_msp", "company_size", "company_okved2", "els", "is_otpr", "is_poluch", "index_pl", "index_fin", "kontrakt_type"], axis=1)

In [ ]:
excel_data_MC["is_msp"] = excel_data_MC["is_msp"].apply(lambda x: 1 if x == "Да" else 0)
excel_data_MC["is_otpr"] = excel_data_MC["is_otpr"].apply(lambda x: 1 if x == "Да" else 0) 
excel_data_MC["is_poluch"] = excel_data_MC["is_poluch"].apply(lambda x: 1 if x == "Да" else 0) 

In [ ]:
excel_data_MC["company_size"] = excel_data_MC["company_size"].apply(lambda x: "Неизвестно" if x is np.nan else x)
excel_data_MC["company_okved2"] = excel_data_MC["company_okved2"].apply(lambda x: "0" if x is np.nan else x)
excel_data_MC["els"] = excel_data_MC["els"].apply(lambda x: 0 if x is np.nan else 1)
excel_data_MC["index_pl"] = excel_data_MC["index_pl"].apply(lambda x: "Недостаточно данных для расчета риска просрочки платежа" if x is np.nan else x)
excel_data_MC["index_fin"] = excel_data_MC["index_fin"].apply(lambda x: "Недостаточно данных для расчета индекса финансового риска" if x is np.nan else x)
excel_data_MC["kontrakt_type"] = excel_data_MC["kontrakt_type"].apply(lambda x: "Неизвестно" if x is np.nan else x)

In [ ]:
company_size = excel_data_MC.loc[:, ["company_size"]].drop_duplicates(ignore_index=True)
company_size.to_excel("././dataset/company_size.xlsx")
index_pl = excel_data_MC.loc[:, ["index_pl"]].drop_duplicates(ignore_index=True)
index_pl.to_excel("././dataset/index_pl.xlsx")
index_fin = excel_data_MC.loc[:, ["index_fin"]].drop_duplicates(ignore_index=True)
index_fin.to_excel("././dataset/index_fin.xlsx")
kontrakt_type = excel_data_MC.loc[:, ["kontrakt_type"]].drop_duplicates(ignore_index=True)
kontrakt_type.to_excel("././dataset/kontrakt_type.xlsx")

In [ ]:
excel_data_MC.loc[:, ["company_size"]] = excel_data_MC.loc[:, ["company_size"]].apply(lambda x: company_size[company_size["company_size"] == x["company_size"]].index[0], axis=1)
excel_data_MC.loc[:, ["index_pl"]] = excel_data_MC.loc[:, ["index_pl"]].apply(lambda x: index_pl[index_pl["index_pl"] == x["index_pl"]].index[0], axis=1)
excel_data_MC.loc[:, ["index_fin"]] = excel_data_MC.loc[:, ["index_fin"]].apply(lambda x: index_fin[index_fin["index_fin"] == x["index_fin"]].index[0], axis=1)
excel_data_MC.loc[:, ["kontrakt_type"]] = excel_data_MC.loc[:, ["kontrakt_type"]].apply(lambda x: kontrakt_type[kontrakt_type["kontrakt_type"] == x["kontrakt_type"]].index[0], axis=1)

In [ ]:
excel_data_MC = excel_data_MC.drop_duplicates()

In [ ]:
company_okved2 = excel_data_MC.groupby("id").apply(lambda x: list(set(list(x["company_okved2"]))), include_groups=False)
company_okved2 = company_okved2.reset_index().set_axis(["id", "company_okved2"], axis=1)  

In [ ]:
is_msp = excel_data_MC.groupby("id").apply(lambda x: max(x["is_msp"]), include_groups=False)
is_msp = is_msp.reset_index().set_axis(["id", "is_msp"], axis=1) 

In [ ]:
company_size = excel_data_MC.groupby("id").apply(lambda x: max(x["company_size"]), include_groups=False)
company_size = company_size.reset_index().set_axis(["id", "company_size"], axis=1)     

In [ ]:
els = excel_data_MC.groupby("id").apply(lambda x: max(x["els"]), include_groups=False)
els = els.reset_index().set_axis(["id", "els"], axis=1)  

In [ ]:
is_otpr = excel_data_MC.groupby("id").apply(lambda x: max(x["is_otpr"]), include_groups=False)
is_otpr = is_otpr.reset_index().set_axis(["id", "is_otpr"], axis=1)  

In [ ]:
is_poluch = excel_data_MC.groupby("id").apply(lambda x: max(x["is_poluch"]), include_groups=False)
is_poluch = is_poluch.reset_index().set_axis(["id", "is_poluch"], axis=1) 

In [ ]:
index_pl = excel_data_MC.groupby("id").apply(lambda x: max(x["index_pl"]), include_groups=False)
index_pl = index_pl.reset_index().set_axis(["id", "index_pl"], axis=1) 

In [ ]:
index_fin = excel_data_MC.groupby("id").apply(lambda x: max(x["index_fin"]), include_groups=False)
index_fin = index_fin.reset_index().set_axis(["id", "index_fin"], axis=1) 

In [ ]:
kontrakt_type = excel_data_MC.groupby("id").apply(lambda x: list(set(list(x["kontrakt_type"]))), include_groups=False)
kontrakt_type = kontrakt_type.reset_index().set_axis(["id", "kontrakt_type"], axis=1) 

In [ ]:
id = excel_data_MC["id"].drop_duplicates()
id = id.reset_index().drop(columns=["index"])

In [ ]:
excel_data_MC.info()

In [ ]:
excel_data = pd.merge(id, is_msp, left_on="id", right_on="id")


In [ ]:
excel_data = pd.merge(excel_data, company_size, left_on="id", right_on="id")

In [ ]:
excel_data = pd.merge(excel_data, company_okved2, left_on="id", right_on="id")

In [ ]:
excel_data = pd.merge(excel_data, els, left_on="id", right_on="id")

In [ ]:
excel_data = pd.merge(excel_data, is_otpr, left_on="id", right_on="id")

In [ ]:
excel_data = pd.merge(excel_data, is_poluch, left_on="id", right_on="id")

In [ ]:
excel_data = pd.merge(excel_data, index_pl, left_on="id", right_on="id")

In [ ]:
excel_data = pd.merge(excel_data, index_fin, left_on="id", right_on="id")

In [ ]:
excel_data = pd.merge(excel_data, kontrakt_type, left_on="id", right_on="id")

In [ ]:
excel_data_MC = excel_data

In [ ]:
excel_data_MC.to_excel("././dataset/MC.xlsx")

In [ ]:
excel_data_MC.info()

In [ ]:
data_pd = excel_data_volumes.drop("subj_from", axis = 1)
data_pd = data_pd.drop("subj_to", axis = 1)
data_pd = data_pd.drop("cargo_code", axis = 1)
data_pd = data_pd.drop("cargo_type", axis = 1)


In [ ]:
data_pd = data_pd.groupby(["id", "month_of_period", "year", "quarter", "month" ], as_index=False).sum()

In [ ]:
data_pd["innactive"] = 0
data_pd["active"] = 0
data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_sales"] == 0)) else 0, axis=1)
data_pd["active"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] > 0) & (x["cargo_sales"] > 0)) else 0, axis=1)

In [ ]:
data_pd.head(10)

In [ ]:
data_pd["outflow"] = 0

def calc_innactive(row):
    prev_row_value = 0
    row_value = data_pd.iloc[row.name]["innactive"]
    row_id = data_pd.iloc[row.name]["id"]
    if row.name > 0:
        prev_row_id = data_pd.iloc[row.name - 1]["id"]
        if row_id == prev_row_id:
            if  row_value > 0:
                prev_row_value = data_pd.iloc[row.name - 1]["innactive"]
                data_pd.iloc[row.name, data_pd.columns.get_loc("innactive")] = prev_row_value + 1
                row_value = data_pd.iloc[row.name]["innactive"]
                if row_value > 11:
                    prev_row_value = 1
                else:
                    prev_row_value = row_value/12
            else: 
                prev_row_value = 0
    return prev_row_value

def calc_active(row):
    row_value = data_pd.iloc[row.name]["active"]
    row_id = data_pd.iloc[row.name]["id"]
    if row.name > 0:
        prev_row_id = data_pd.iloc[row.name - 1]["id"]
        if row_id == prev_row_id:
            if  row_value > 0:
                prev_row_value = data_pd.iloc[row.name - 1]["active"]
                data_pd.iloc[row.name, data_pd.columns.get_loc("active")] = prev_row_value + 1
    return 0

data_pd["outflow"] = data_pd.apply(calc_innactive, axis=1)
# data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_sales"] == 0)) else 0, axis=1)
data_pd.apply(calc_active, axis=1)


In [ ]:
data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_sales"] == 0)) else 0, axis=1)
data_pd["active"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] > 0) & (x["cargo_sales"] > 0)) else 0, axis=1)

In [ ]:
excel_data = data_pd[data_pd["month_of_period"] == max(data_pd["month_of_period"])].loc[:, ["id", "outflow"]].reset_index()

In [ ]:
excel_data = excel_data.drop(columns = ["index"])

In [ ]:
excel_data["outflow"] = excel_data["outflow"].apply(lambda x: 1 if x ==1 else 0)

In [ ]:
data_pd = data_pd.drop(columns = ["month_of_period", "year", "quarter", "month", "outflow"]).groupby(["id"]).sum()

In [ ]:
data_pd["activity"] = data_pd.apply(lambda x: x["active"]/(x["active"]+x["innactive"]), axis = 1)

In [ ]:
data_pd = data_pd.drop(columns=["innactive", "active"])

In [ ]:
data_pd

In [ ]:
data_pd = pd.merge(data_pd, excel_data, left_on="id", right_on="id")

In [ ]:
data_pd = data_pd.drop(columns=["cargo_price"])

In [ ]:
data_pd = data_pd.rename(columns = {"outflow" : "out"})

In [ ]:
data_pd

In [ ]:
excel_data_MC

In [ ]:
excel_data = pd.merge(excel_data_MC, data_pd, left_on="id", right_on="id", how="outer")

In [ ]:
excel_data

In [ ]:
excel_data = excel_data[excel_data["out"].notna()]

In [ ]:
excel_data.loc[:, ["is_msp"]] = excel_data.loc[:, ["is_msp"]].fillna(0).astype("int64")

In [ ]:
excel_data.loc[:, ["company_size"]] = excel_data.loc[:, ["company_size"]].fillna(0).astype("int64")

In [ ]:
excel_data.loc[:, ["company_okved2"]] = excel_data.loc[:, ["company_okved2"]].fillna(0).apply(lambda x: x["company_okved2"] if x["company_okved2"] !=0 else [0], axis =1)

In [ ]:
excel_data.loc[:, ["els"]] = excel_data.loc[:, ["els"]].fillna(0).astype("int64")

In [ ]:
excel_data.loc[:, ["is_otpr"]] = excel_data.loc[:, ["is_otpr"]].fillna(1).astype("int64")

In [ ]:
excel_data.loc[:, ["is_poluch"]] = excel_data.loc[:, ["is_poluch"]].fillna(0).astype("int64")

In [ ]:
excel_data.loc[:, ["index_pl"]] = excel_data.loc[:, ["index_pl"]].fillna(0).astype("int64")

In [ ]:
excel_data.loc[:, ["index_fin"]] = excel_data.loc[:, ["index_fin"]].fillna(0).astype("int64")

In [ ]:
excel_data.loc[:, ["kontrakt_type"]] = excel_data.loc[:, ["kontrakt_type"]].fillna(0).apply(lambda x: x["kontrakt_type"] if x["kontrakt_type"] !=0 else [0], axis =1)

In [ ]:
excel_data = excel_data.convert_dtypes()

In [ ]:
excel_data = excel_data.reset_index().drop(columns=["index"])

In [ ]:
excel_data.to_excel("././dataset/merge_data.xlsx")

In [ ]:
excel_data

In [ ]:
ProfileReport(excel_data, minimal = True) # Формирование отчета для исследования данных

In [ ]:
excel_data.groupby(by = "out").mean() # расчет средних значений признаков для тех кто ушел и остался

In [ ]:
cor = excel_data.corr() # матрица корреляций
fig, ax = plt.subplots(figsize=(15,5)) 
sns.heatmap(cor, annot = True, fmt = ".2f") # тепловая карта на основе матрицы корреляций

In [ ]:
keep = excel_data['id'].drop_duplicates().sample(frac=0.7)
m = excel_data['id'].isin(keep)
train = excel_data[m]
test = excel_data[~m]

train.to_excel("train.xlsx")
test.to_excel("test.xlsx")

In [ ]:
X_train = train.drop("out", axis = 1)
y_train = train["out"] # разделение данных на признаки (матрица X) и целевую переменную (y)
X_test = test.drop("out", axis = 1).drop("out", axis = 1)
y_test = test["out"] # разделение данных на признаки (матрица X) и целевую переменную (y)

X = pd.concat([X_train, X_test], axis=0)

In [ ]:
model = LogisticRegression(max_iter = 1000)  
model.fit(X_train,y_train)
predictions = model.predict(X_test)
probabilities = model.predict_proba(X_test)[:,1]
acc = accuracy_score(y_test, predictions)
precision = precision_score (y_test, predictions)
recall = recall_score (y_test, predictions)
print(acc, precision, recall) # вывод на экран метрик, характеризующих качество модели

In [ ]:
rf_model = RandomForestClassifier(n_estimators = 100)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test) 
acc_rf = accuracy_score(y_test, y_pred)
precision_rf = precision_score (y_test, y_pred)
recall_rf = recall_score (y_test, y_pred)
print(acc_rf, precision_rf, recall_rf)# вывод на экран метрик, характеризующих качество модели

In [ ]:
sc = StandardScaler()
X_sc = sc.fit_transform(X)
linked = linkage(X_sc, method = "ward")
plt.figure(figsize=(15, 10))  
dendrogram(linked, orientation="top")
plt.show() # вывод на экран дендрограммы